In [1]:
import pandas as pd
import requests
import numpy as np
import json
import sqlite3
from sqlite3 import Error

In [2]:
id = 91

In [3]:
r = requests.get('https://lcid.cc/info/{}'.format(str(id)))
data = r.json()
df = pd.DataFrame.from_dict(data, orient='columns')

In [4]:
df = df.drop('acRate', 1)
df = df.drop('freqBar', 1)
df = df.drop('isFavor', 1)
df = df.drop('paidOnly', 1)
df = df.drop('status', 1)
df = df.drop('hasSolution', 1)
df = df.drop('hasVideoSolution', 1)

In [5]:
df.rename(columns={'frontendQuestionId': 'id'}, inplace=True)

In [6]:
xz = df.join(pd.json_normalize(df["topicTags"].tolist()).add_prefix("topicTags")).drop(["topicTags"], axis=1)

In [7]:
xz.head()

,difficulty,id,title,titleSlug,topicTagsname,topicTagsid,topicTagsslug
0,Medium,91,Decode Ways,decode-ways,String,VG9waWNUYWdOb2RlOjEw,string
1,Medium,91,Decode Ways,decode-ways,Dynamic Programming,VG9waWNUYWdOb2RlOjEz,dynamic-programming


In [9]:
DB_FILE_PATH = 'sampleSQLite.db'

In [10]:
def connect_to_db(db_file):
    """
    Connect to an SQlite database, if db file does not exist it will be created
    :param db_file: absolute or relative path of db file
    :return: sqlite3 connection
    """
    sqlite3_conn = None

    try:
        sqlite3_conn = sqlite3.connect(db_file)
        return sqlite3_conn

    except Error as err:
        print(err)

        if sqlite3_conn is not None:
            sqlite3_conn.close()

conn = connect_to_db(DB_FILE_PATH)
if conn is not None:
    c = conn.cursor()
    c.execute('CREATE TABLE IF NOT EXISTS ' + 'random' +
                  '(difficulty        VARCHAR,'
                  'id        VARCHAR,'
                  'title        VARCHAR,'
                  'titleSlug  VARCHAR,'
                  'topicTagsname    VARCHAR,'
                  'topicTagsid       VARCHAR,'
                  'topicTagsslug VARCHAR)')

    xz.to_sql(name="random", con=conn, if_exists='append', index=False)

    conn.close()
    print('SQL insert process finished')
else:
    print('Connection to database failed')

SQL insert process finished
